# Deep Learning Project - Gesture Recognition

## Problem Statement

Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

## Objectives:

Generator: The generator should be able to take a batch of videos as input without any error. Steps like cropping, resizing and normalization should be performed successfully.

Model: Develop a model that is able to train without any errors which will be judged on the total number of parameters (as the inference(prediction) time should be less) and the accuracy achieved. As suggested by Snehansu, start training on a small amount of data and then proceed further.

Write up: This should contain the detailed procedure followed in choosing the final model. The write up should start with the reason for choosing the base model, then highlight the reasons and metrics taken into consideration to modify and experiment to arrive at the final model.


#### Importing the Libraries

In [2]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
from skimage.transform import resize

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [4]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

#write your model here

#### Model a

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

2022-08-23 01:09:10.412617: W tensorflow/stream_executor/platform/default/dso_loader.cc:65] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-08-23 01:09:10.412666: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-23 01:09:10.412694: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist


In [9]:
optimiser = tf.keras.optimizers.Adam(lr=0.001) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 8)  32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 8)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_1 (Batc  (None, 15, 60, 60, 16)   6

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_540/1387693686.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 4.8874 - categorical_accuracy: 0.2715 Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-08-2104_11_02.784007/model-00001-4.88739-0.27149-1.85692-0.32000.h5
67/67 [==============================] - 931s 14s/step - loss: 4.8874 - categorical_accuracy: 0.2715 - val_loss: 1.8569 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.2614 - categorical_accuracy: 0.3085
Epoch 00002: saving model to model_init_2022-08-2104_11_02.784007/model-00002-2.26137-0.30846-2.05242-0.27000.h5
67/67 [==============================] - 332s 5s/step - loss: 2.2614 - categorical_accuracy: 0.3085 - val_loss: 2.0524 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.9342 - categorical_accur

In [13]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [14]:
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

#### Model b

In [15]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 120, 120, 32)  2624      
                                                                 
 activation_4 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 conv3d_5 (Conv3D)           (None, 30, 120, 120, 32)  27680     
                                                                 
 activation_5 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 10, 40, 40, 32)   0         
 3D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 10, 40, 40, 32)    0         
                                                      

In [16]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/10


In [17]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_9 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 10, 20, 20, 32)   0         
 3D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [18]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [32]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6108 - categorical_accuracy: 0.2247 Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-08-2111_10_16.876385/model-00001-1.61082-0.22474-1.47694-0.37000.h5
67/67 [==============================] - 1242s 19s/step - loss: 1.6108 - categorical_accuracy: 0.2247 - val_loss: 1.4769 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.5575 - categorical_accuracy: 0.2587
Epoch 00002: saving model to model_init_2022-08-2111_10_16.876385/model-00002-1.55750-0.25871-1.51619-0.20000.h5
67/67 [==============================] - 441s 7s/step - loss: 1.5575 - categorical_accuracy: 0.2587 - val_loss: 1.5162 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.5387 - categorical_accu

In [19]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [20]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 20
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.6174 - categorical_accuracy: 0.7454 Source path =  /home/datasets/Project_data/val ; batch size = 20

Epoch 00001: saving model to model_init_2022-08-2106_33_24.710306/model-00001-0.61740-0.74541-0.95090-0.69000.h5
67/67 [==============================] - 1517s 22s/step - loss: 0.6174 - categorical_accuracy: 0.7454 - val_loss: 0.9509 - val_categorical_accuracy: 0.6900 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.5890 - categorical_accuracy: 0.8209
Epoch 00002: saving model to model_init_2022-08-2106_33_24.710306/model-00002-0.58903-0.82090-0.62669-0.77500.h5
67/67 [==============================] - 499s 7s/step - loss: 0.5890 - categorical_accuracy: 0.8209 - val_loss: 0.6267 - val_categorical_accuracy: 0.7750 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.5871 - categorical_accu

In [20]:
train_generator = generator_ex(train_path, train_doc, 10)
val_generator = generator_ex(val_path, val_doc, 10)

In [24]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6108 - categorical_accuracy: 0.2247 Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-08-2112_46_17.878532/model-00001-1.61082-0.22474-1.47694-0.37000.h5
67/67 [==============================] - 1302s 19s/step - loss: 1.6108 - categorical_accuracy: 0.2247 - val_loss: 1.4769 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.5575 - categorical_accuracy: 0.2587
Epoch 00002: saving model to model_init_2022-08-2112_46_17.878532/model-00002-1.55750-0.25871-1.51619-0.20000.h5
67/67 [==============================] - 441s 7s/step - loss: 1.5575 - categorical_accuracy: 0.2587 - val_loss: 1.5162 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.5387 - categorical_accu

In [21]:
train_generator = generator_ex(train_path, train_doc, 15)
val_generator = generator_ex(val_path, val_doc, 15)

In [ ]:
model_b.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 15
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.5647 - categorical_accuracy: 0.7860 Source path =  /home/datasets/Project_data/val ; batch size = 15
Batch:  7 Index: 15

Epoch 00001: saving model to model_init_2022-08-2112_46_17.878532/model-00001-0.56474-0.78601-0.89761-0.71538.h5
67/67 [==============================] - 1373s 20s/step - loss: 0.5647 - categorical_accuracy: 0.7860 - val_loss: 0.8976 - val_categorical_accuracy: 0.7154 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.6292 - categorical_accuracy: 0.7562
Epoch 00002: saving model to model_init_2022-08-2112_46_17.878532/model-00002-0.62918-0.75622-0.83850-0.73000.h5
67/67 [==============================] - 421s 6s/step - loss: 0.6292 - categorical_accuracy: 0.7562 - val_loss: 0.8385 - val_categorical_accuracy: 0.7300 - lr: 0.0010
Epoch 3/10
19/67 [=======>......................] - ETA: 4:16 - loss: 0.5

In [22]:
model_b.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_9 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 10, 20, 20, 32)   0         
 3D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [23]:
train_generator = generator_ex(train_path, train_doc, 16)
val_generator = generator_ex(val_path, val_doc, 16)

In [21]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_453/3161962229.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 16
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6511 - categorical_accuracy: 0.1838 Source path =  /home/datasets/Project_data/val ; batch size = 16
Batch:  7 Index: 16

Epoch 00001: saving model to model_init_2022-08-2115_22_44.027774/model-00001-1.65113-0.18377-1.60919-0.24107.h5
67/67 [==============================] - 1518s 23s/step - loss: 1.6511 - categorical_accuracy: 0.1838 - val_loss: 1.6092 - val_categorical_accuracy: 0.2411 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6291 - categorical_accuracy: 0.2026 
Epoch 00002: saving model to model_init_2022-08-2115_22_44.027774/model-00002-1.62913-0.20256-1.60805-0.22500.h5
67/67 [==============================] - 832s 12s/step - loss: 1.6291 - categorical_accuracy: 0.2026 - val_loss: 1.6080 - val_categorical_accuracy: 0.2250 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6

In [24]:
num_epochs = 18 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 18


In [25]:
import keras
model_b.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_9 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 10, 20, 20, 32)   0         
 3D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [26]:
train_generator = generator_ex(train_path, train_doc, 18)
val_generator = generator_ex(val_path, val_doc, 18)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_453/3161962229.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 18
Epoch 1/18
67/67 [==============================] - ETA: 0s - loss: 1.6057 - categorical_accuracy: 0.2120 Source path =  /home/datasets/Project_data/val ; batch size = 18
Batch:  6 Index: 18

Epoch 00001: saving model to model_init_2022-08-2115_22_44.027774/model-00001-1.60574-0.21204-1.58575-0.25714.h5
67/67 [==============================] - 2063s 31s/step - loss: 1.6057 - categorical_accuracy: 0.2120 - val_loss: 1.5857 - val_categorical_accuracy: 0.2571 - lr: 0.0010
Epoch 2/18
67/67 [==============================] - ETA: 0s - loss: 1.6114 - categorical_accuracy: 0.2005
Epoch 00002: saving model to model_init_2022-08-2115_22_44.027774/model-00002-1.61138-0.20054-1.57708-0.31000.h5
67/67 [==============================] - 738s 11s/step - loss: 1.6114 - categorical_accuracy: 0.2005 - val_loss: 1.5771 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 3/18
67/67 [==============================] - ETA: 0s - loss: 1.60

#### Model c

In [27]:
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(classes, activation='softmax'))
model_c.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_12 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_13 (Conv3D)          (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_13 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 10, 20, 20, 32)   0         
 3D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [28]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120# image height
num_epochs = 20

In [29]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

#### Model d

In [30]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_18 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_5 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_18 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_11 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_19 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_6 (Batc  (None, 15, 60, 60, 16)  

In [58]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_450/3797531445.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.8321 - categorical_accuracy: 0.2308 Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-08-2214_21_51.947313/model-00001-2.83211-0.23077-1.60487-0.25000.h5
67/67 [==============================] - 929s 14s/step - loss: 2.8321 - categorical_accuracy: 0.2308 - val_loss: 1.6049 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.6786 - categorical_accuracy: 0.2289
Epoch 00002: saving model to model_init_2022-08-2214_21_51.947313/model-00002-2.67861-0.22886-1.61781-0.17000.h5
67/67 [==============================] - 322s 5s/step - loss: 2.6786 - categorical_accuracy: 0.2289 - val_loss: 1.6178 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.9434 - categorical_accur

#### Model e

In [31]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

In [32]:
model_e = Sequential()
model_e.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_e.add(Activation('relu'))
model_e.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Flatten())
model_e.add(Dense(512, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(classes, activation='softmax'))
model_e.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_22 (Conv3D)          (None, 30, 64, 64, 32)    896       
                                                                 
 activation_22 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 conv3d_23 (Conv3D)          (None, 30, 64, 64, 32)    27680     
                                                                 
 activation_23 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 max_pooling3d_15 (MaxPoolin  (None, 10, 22, 22, 32)   0         
 g3D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 10, 22, 22, 32)    0         
                                                      

In [61]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 5


/tmp/ipykernel_450/1728685948.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.1117 - categorical_accuracy: 0.1672Source path =  /home/datasets/Project_data/val ; batch size = 5

Epoch 00001: saving model to model_init_2022-08-2214_21_51.947313/model-00001-2.11169-0.16716-1.61322-0.12000.h5
67/67 [==============================] - 699s 10s/step - loss: 2.1117 - categorical_accuracy: 0.1672 - val_loss: 1.6132 - val_categorical_accuracy: 0.1200 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.1338 - categorical_accuracy: 0.1420 
Epoch 00002: saving model to model_init_2022-08-2214_21_51.947313/model-00002-2.13376-0.14199-1.61651-0.18000.h5
67/67 [==============================] - 699s 10s/step - loss: 2.1338 - categorical_accuracy: 0.1420 - val_loss: 1.6165 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.0149 - categorical_accuracy: 0.1692
Epoch 00003: saving model to model_init_2022-08-2214_21

#### Model f

In [33]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_28 (Conv3D)          (None, 30, 120, 120, 8)   224       
                                                                 
 batch_normalization_10 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_18 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_29 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_11 (Bat  (None, 15, 60, 60, 16)  

In [63]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_450/3462510936.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.9674 - categorical_accuracy: 0.2232 Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-08-2214_21_51.947313/model-00001-2.96743-0.22323-1.62591-0.17000.h5
67/67 [==============================] - 827s 12s/step - loss: 2.9674 - categorical_accuracy: 0.2232 - val_loss: 1.6259 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.8714 - categorical_accuracy: 0.2438
Epoch 00002: saving model to model_init_2022-08-2214_21_51.947313/model-00002-2.87136-0.24378-1.62605-0.23000.h5
67/67 [==============================] - 292s 4s/step - loss: 2.8714 - categorical_accuracy: 0.2438 - val_loss: 1.6261 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.9492 - categorical_accuracy: 0.1940
Epoch 00003: saving model to model_init_2022-08-2214_21

#### Model g

In [34]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_32 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_14 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_32 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_22 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_33 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_15 (Bat  (None, 15, 60, 60, 16)  

In [35]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_429/2187855109.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /home/datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 3.2120 - categorical_accuracy: 0.1991 Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-08-2301_09_04.588158/model-00001-3.21198-0.19910-1.65661-0.24000.h5
67/67 [==============================] - 893s 13s/step - loss: 3.2120 - categorical_accuracy: 0.1991 - val_loss: 1.6566 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 3.2570 - categorical_accuracy: 0.2090
Epoch 00002: saving model to model_init_2022-08-2301_09_04.588158/model-00002-3.25699-0.20896-1.70172-0.21000.h5
67/67 [==============================] - 316s 5s/step - loss: 3.2570 - categorical_accuracy: 0.2090 - val_loss: 1.7017 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 3.1276 - categorical_accur

#### Model h

In [36]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_h = Sequential()

model_h.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_h.add(Flatten())

model_h.add(Dense(nb_dense[0], activation='relu'))
model_h.add(Dropout(0.5))

model_h.add(Dense(nb_dense[1], activation='relu'))
model_h.add(Dropout(0.5))

#softmax layer
model_h.add(Dense(nb_dense[2], activation='softmax'))
model_h.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_h.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_36 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_18 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_36 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_26 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_37 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_19 (Bat  (None, 15, 60, 60, 16)  

#### Model final

In [37]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [38]:
optimiser = tf.keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_40 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_22 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_40 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_30 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_41 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_23 (Bat  (None, 15, 60, 60, 16)  

In [39]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [40]:
batch_size = 10
num_epochs = 12
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 10


/tmp/ipykernel_429/655006215.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/12
67/67 [==============================] - ETA: 0s - loss: 2.0108 - categorical_accuracy: 0.2262 Source path =  /home/datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-08-2301_09_04.588158/model-00001-2.01084-0.22624-1.58078-0.35000.h5
67/67 [==============================] - 890s 13s/step - loss: 2.0108 - categorical_accuracy: 0.2262 - val_loss: 1.5808 - val_categorical_accuracy: 0.3500 - lr: 0.0010
Epoch 2/12
67/67 [==============================] - ETA: 0s - loss: 1.6030 - categorical_accuracy: 0.2438
Epoch 00002: saving model to model_init_2022-08-2301_09_04.588158/model-00002-1.60297-0.24378-1.59883-0.17000.h5
67/67 [==============================] - 315s 5s/step - loss: 1.6030 - categorical_accuracy: 0.2438 - val_loss: 1.5988 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 3/12
67/67 [==============================] - ETA: 0s - loss: 1.6164 - categorical_accuracy: 0.2239
Epoch 00003: saving model to model_init_2022-08-2301_09

In [41]:
batch_size = 12
num_epochs = 14
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_429/3909543589.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/14
67/67 [==============================] - ETA: 0s - loss: 1.3186 - categorical_accuracy: 0.4627
Epoch 00001: saving model to model_init_2022-08-2301_09_04.588158/model-00001-1.31864-0.46269-1.12038-0.59000.h5
67/67 [==============================] - 315s 5s/step - loss: 1.3186 - categorical_accuracy: 0.4627 - val_loss: 1.1204 - val_categorical_accuracy: 0.5900 - lr: 5.0000e-04
Epoch 2/14
67/67 [==============================] - ETA: 0s - loss: 1.2876 - categorical_accuracy: 0.4577
Epoch 00002: saving model to model_init_2022-08-2301_09_04.588158/model-00002-1.28762-0.45771-1.15274-0.55000.h5
67/67 [==============================] - 314s 5s/step - loss: 1.2876 - categorical_accuracy: 0.4577 - val_loss: 1.1527 - val_categorical_accuracy: 0.5500 - lr: 5.0000e-04
Epoch 3/14
67/67 [==============================] - ETA: 0s - loss: 1.3071 - categorical_accuracy: 0.4826
Epoch 00003: saving model to model_init_2022-08-2301_09_04.588158/model-00003-1.30715-0.48259-1.31195-0.54000.h5

